In [1]:
import numpy as np

In [2]:
S = np.array([[1, 0.4508],[0.4508,1]])
print(S)
H_core = np.array([[-2.6527, -1.3472],[-1.3472, -1.7318]])
print(H_core)

[[1.     0.4508]
 [0.4508 1.    ]]
[[-2.6527 -1.3472]
 [-1.3472 -1.7318]]


In [3]:
print(1/np.sqrt(2))

0.7071067811865475


In [4]:
U = np.array([[0.707, 0.707],[0.707, -0.707]])

In [5]:
def create_unitary_from_overlap(overlap_matrix):
    U = np.linalg.eig(overlap_matrix)[1]
    return U

def create_s_half(overlap_matrix, unitary_transformation):
    s = np.round(unitary_transformation.T@overlap_matrix@unitary_transformation,4)
    s_diagonal = 1/np.sqrt(np.diagonal(s))
    s_half = np.diag(s_diagonal)
    return s_half

def canonical_transformation(unitary_transformation, s_half):
    X_can = unitary_transformation@s_half
    return X_can
    
U = create_unitary_from_overlap(S)
print(U)
s_half = create_s_half(S, U)
print(s_half)
X_can = canonical_transformation(U, s_half)
print(X_can)

[[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]]
[[0.8302258  0.        ]
 [0.         1.34938145]]
[[ 0.58705829 -0.95415677]
 [ 0.58705829  0.95415677]]


## storing two-electron integrals forms a necessary part of the calculations. In case of real orbitals, inherent symmetry of the two electron integrals leads to a reduction in the number of in-equivalent integrals (see szabo ostlund example 3.14). Also, look at Yoshimine sorting on wikipedia to store the two electron integrals.

In [6]:
## lets create an order 4 tensor to save the two electron integrals. NOTE: this might not be a memory efficient way to do it
two_electron_integrals = np.zeros((2,2,2,2))
print(two_electron_integrals.shape)

(2, 2, 2, 2)


In [7]:
two_electron_integrals[0,0,0,0] = 1.3072
two_electron_integrals[1,1,1,1] = 0.7746
two_electron_integrals[0,0,1,1] = 0.6057
two_electron_integrals[1,1,0,0] = 0.6057
two_electron_integrals[0,0,0,1] = 0.4373
two_electron_integrals[0,0,1,0] = 0.4373
two_electron_integrals[1,0,0,0] = 0.4373
two_electron_integrals[0,1,0,0] = 0.4373

two_electron_integrals[1,1,1,0] = 0.3118
two_electron_integrals[1,1,0,1] = 0.3118
two_electron_integrals[1,0,1,1] = 0.3118
two_electron_integrals[0,1,1,1] = 0.3118
two_electron_integrals[1,0,1,0] = 0.1773
two_electron_integrals[1,0,0,1] = 0.1773
two_electron_integrals[0,1,1,0] = 0.1773
two_electron_integrals[0,1,0,1] = 0.1773

In [8]:
print(two_electron_integrals)

[[[[1.3072 0.4373]
   [0.4373 0.6057]]

  [[0.4373 0.1773]
   [0.1773 0.3118]]]


 [[[0.4373 0.1773]
   [0.1773 0.3118]]

  [[0.6057 0.3118]
   [0.3118 0.7746]]]]


In [9]:
##starting iterations

In [12]:
# i = 1
def solve_fock_equation(H_core, G, X_can):
    F = H_core + G
    F_dash = X_can.T@F@X_can
    epsilon, C_dash = np.linalg.eig(F_dash)
    epsilon = np.diag(epsilon)
    C = X_can@C_dash
    return C, epsilon, F

def create_density_matrix_from_coefficients(C):
    number_of_occupied_orbitals = int((C.shape[0])/2)
    P = 2*C[:, :number_of_occupied_orbitals].reshape((-1,1))@C[:, :number_of_occupied_orbitals].reshape((1,-1))
    return P

def compute_G_matrix(H_core, P, two_electron_integrals):
    G = np.zeros(F.shape)
    for u in range(F.shape[0]):
        for v in range(F.shape[1]):
            temp_term = 0
            for w in range(two_electron_integrals.shape[2]):
                for x in range(two_electron_integrals.shape[3]):
                    temp_term += P[w,x]*(two_electron_integrals[u,v,w,x]-(0.5*two_electron_integrals[u,x,w,v]))
            G[u,v] = temp_term
    return G
    
    
def compute_total_energy(P, H_core, F):
    E = 0.5*(np.sum(P.T*(H_core + F)))
    return E
    
for i in range(1,7):
    if i==1:
        C, epsilon, F = solve_fock_equation(H_core, 0, X_can)
        P = create_density_matrix_from_coefficients(C)
        print('P = ', P)
    else:
        G = compute_G_matrix(H_core, P, two_electron_integrals)
        C, epsilon, F = solve_fock_equation(H_core, G, X_can)
        E = compute_total_energy(P, H_core, F)
        print(f'Total energy at step {i-1} = {E}')
        P = create_density_matrix_from_coefficients(C)
        print('P = ', P)

G = compute_G_matrix(H_core, P, two_electron_integrals)
C, epsilon, F = solve_fock_equation(H_core, G, X_can)
E = compute_total_energy(P, H_core, F)
print(f'Final Total energy = {E}')

P =  [[1.72671245 0.25976884]
 [0.25976884 0.03907996]]
Total energy at step 1 = -4.141673308407317
P =  [[1.3341882  0.51661004]
 [0.51661004 0.20003619]]
Total energy at step 2 = -4.226327859264931
P =  [[1.28989512 0.53837478]
 [0.53837478 0.22470618]]
Total energy at step 3 = -4.227358288504301
P =  [[1.2864205  0.54002298]
 [0.54002298 0.22669478]]
Total energy at step 4 = -4.227364619125003
P =  [[1.28615921 0.54014657]
 [0.54014657 0.22684464]]
Total energy at step 5 = -4.227364654921209
P =  [[1.28613963 0.54015584]
 [0.54015584 0.22685587]]
Final Total energy = -4.227364655122287
